In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('sredjeni_kurs.csv', parse_dates=True)
print(df.head())

  APPLICATION DATE  MIDDLE EXCHANGE RATE
0       2003-01-01               61.5845
1       2003-01-02               61.5845
2       2003-01-03               61.5845
3       2003-01-04               61.6615
4       2003-01-05               61.6615


In [4]:
from sklearn.model_selection import train_test_split

# Podela podataka na testni i trening skup
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

In [5]:
from pmdarima.arima import auto_arima
import pandas as pd

# Definisanje i treniranje modela
model = auto_arima(train_data['MIDDLE EXCHANGE RATE'], trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train_data['MIDDLE EXCHANGE RATE'])

# Predviđanja za testni skup
forecast = model.predict(n_periods=len(test_data))
forecast = pd.DataFrame(forecast, index=test_data.index, columns=['Prediction'])


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-168.547, Time=4.33 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=234.765, Time=0.52 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-124.910, Time=0.80 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-158.968, Time=0.59 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=241.175, Time=0.20 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-170.844, Time=2.32 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-159.586, Time=0.76 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-160.494, Time=1.08 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-168.933, Time=4.01 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-164.339, Time=0.86 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-164.074, Time=1.66 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-166.887, Time=4.51 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-164.843, Time=0.83 sec

Best model:  ARIMA(1,1,2)(0,0,0)[0] intercept
Total fit time: 22.475 seconds


Ovo ukazuje na to da ARIMA(1,1,2)(0,0,0)[0] model najbolje predviđa srednji kurs evra.

In [6]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

mse = mean_squared_error(test_data['MIDDLE EXCHANGE RATE'], forecast['Prediction'])

mae = mean_absolute_error(test_data['MIDDLE EXCHANGE RATE'], forecast['Prediction'])

rmse = np.sqrt(mse)


# Ispisivanje rezultata
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


Mean Squared Error (MSE): 67.93252047507656
Mean Absolute Error (MAE): 7.119628510271598
Root Mean Squared Error (RMSE): 8.242118688485174
Mean Absolute Percentage Error (MAPE): 6.064534529412018


Podesavanje hiperparametara:

In [7]:
# Definisanje parametara pretrage
stepwise_model = auto_arima(train_data['MIDDLE EXCHANGE RATE'], 
                            start_p=1, start_q=1,
                            max_p=3, max_q=3, 
                            m=12,  # broj vremenskih koraka u jednoj sezoni
                            start_P=0, seasonal=True,
                            d=1, D=1, trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

# Treniranje modela
stepwise_model.fit(train_data['MIDDLE EXCHANGE RATE'])


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=16.10 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=4514.722, Time=0.39 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=2135.298, Time=2.16 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=10.66 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=4168.702, Time=0.90 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=1398.061, Time=4.28 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=37.40 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=19.09 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=1755.000, Time=2.34 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=1369.329, Time=6.10 sec
 ARIMA(2,1,0)(1,1,0)[12]             : AIC=2098.972, Time=3.24 sec
 ARIMA(2,1,0)(2,1,1)[12]             : AIC=inf, Time=40.04 sec
 ARIMA(2,1,0)(1,1,1)[12]             : AIC=inf, Time=27.35 sec
 ARIMA(3,1,0)(2,1,0)[12]             : AIC=1370.849, Time=6.30 sec
 ARIMA(2,1,1)(2,1,0)[12]             : AIC=

ARIMA(order=(2, 1, 1), scoring_args={}, seasonal_order=(2, 1, 0, 12),
      suppress_warnings=True, with_intercept=False)

In [8]:
# Predviđanja za testni skup
forecast_stepwise = stepwise_model.predict(n_periods=len(test_data))
forecast_stepwise = pd.DataFrame(forecast_stepwise, index=test_data.index, columns=['Prediction'])

In [11]:
print(forecast_stepwise)

      Prediction
6136  117.598721
6137  117.593347
6138  117.603107
6139  117.600622
6140  117.597507
...          ...
7665  120.146813
7666  120.138998
7667  120.126702
7668  120.138882
7669  120.158986

[1534 rows x 1 columns]


In [10]:
# Izračunavanje metrika
mse = mean_squared_error(test_data['MIDDLE EXCHANGE RATE'], forecast_stepwise)
mae = mean_absolute_error(test_data['MIDDLE EXCHANGE RATE'], forecast_stepwise)
rmse = np.sqrt(mse)

# Ispis rezultata
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)


Mean Squared Error (MSE): 2.7098264822628897
Mean Absolute Error (MAE): 1.394207234979689
Root Mean Squared Error (RMSE): 1.6461550602123998
